In [100]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [101]:
train = pd.read_csv('train.txt', sep="\t")
test = pd.read_csv('test.txt', sep="\t")
base1 = pd.read_csv('Base1.txt', sep="\t")

In [102]:
# trainBase1 це будуть дані з base1 для юзерів з тренувальнї вибірки

trainBase1 = pd.DataFrame()

for n in train['ID'].values:
    trainBase1 = trainBase1.append(base1[base1['ID'] == n], ignore_index = True)

# пропущені значення заповнимо медіанними
for n in trainBase1:
    trainBase1[n].fillna(trainBase1[n].median(), inplace=True)
    
# згрупуємо всі дані про одного юзера, використовуючи середнє значення 
#!!!!!!!trainBase1_cut_id = trainBase1.groupby(['ID']).mean().reset_index()
trainBase1_df = trainBase1.groupby(['ID']).mean().reset_index()
# посортуємо дані тренувальної вибірки результатів
#!!!!!!trainSort = train.sort_values(['ID']).reset_index()
train_df = train.sort_values(['ID']).reset_index()

# зробимо те ж саме для тестової вибірки

testBase1 = pd.DataFrame()

for n in test['ID'].values:
    testBase1 = testBase1.append(base1[base1['ID'] == n], ignore_index = True)
    
for n in testBase1:
    testBase1[n].fillna(testBase1[n].median(), inplace=True)
    
#!!!!!!testBase1_cut_id = testBase1.groupby(['ID']).mean().reset_index()
testBase1_df = testBase1.groupby(['ID']).mean().reset_index()

#!!!!!!testSort = test.sort_values(['ID']).reset_index()
test_df = test.sort_values(['ID']).reset_index()

In [103]:
# далі побудуємо просте дерево рішень для класифікації користувачів
features = trainBase1_df
y = train_df['TARGET']

features_train, features_test, y_train, y_test = train_test_split(features,y,test_size=0.35)

In [104]:
tree = DecisionTreeClassifier()
tree.fit(features_train,y_train)
y_pred_tree = tree.predict(features_test)

In [110]:
print("Accuracy",accuracy_score(y_test,y_pred_tree))

Accuracy 0.36221916552


In [111]:
test_predlist = []

for n in test_df['ID']:
    x = testBase1_df[testBase1_df['ID']==n]
    test_predlist.append(tree.predict(x))
    

In [112]:
test_df['TARGET'] = test_predlist
test_df['TARGET'] = test_df['TARGET'].astype('int64')

In [113]:
test_df

,index,ID,TARGET
0,2898,7,3
1,1130,11,0
2,2393,12,2
3,300,15,2
4,138,16,1
5,167,17,3
6,1348,18,1
7,188,20,2
8,2742,22,3
9,2501,24,2


In [114]:
# запис у файл
test_df[['ID','TARGET']].to_csv(r'PrysiazhnaLiudmyla_test.txt',index=None,sep='\t')